# The Caring Home / Helpful Home Continued

A model that can look for non normal situations in a elderly person home, from smart home IoT sensors.

## Use AI to scan log file - but with a twist!!

We will use an LLM to interpret the log files and turn this log data into understandable scenarios.
Then we will use another LLM to classify the scenario was normal or anomalous

In [49]:
# imports

import os
import json
import re
from typing import Optional, List
from openai import OpenAI
from agents.situations import LoadedSituation, SituationSelection
from agents.agent import Agent
from datetime import datetime, timedelta
from dotenv import load_dotenv
from anthropic import Anthropic


In [50]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['OPENROUTER_API_KEY'] = os.getenv('OPENROUTER_API_KEY', 'your-key-if-not-using-env')
os.environ['HYPERBOLIC_API_KEY'] = os.getenv('HYPERBOLIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [51]:
openai = OpenAI()
claude = Anthropic()

In [66]:
TIMEZONE = datetime.now().astimezone().tzinfo

SYSTEM_PROMPT = f"The timezone here is {str(TIMEZONE)}. "
SYSTEM_PROMPT += """Your task is to analyze log entries from home sensors from a home occupied by an elderly person by grouping them into 6-hour intervals and converting each grouping into a clear, human-readable scenario. For each 6-hour interval, generate a neutral narrative that describes what occurred during that period. Important: The narrative (under "situation_description") must describe the events without any judgment about whether they are normal or anomalous. The determination of "normal" or "anomalous" should be provided solely in the separate "result" field when you are highly confident.

For each 6-hour grouping, ensure the output includes:

    situation_description: A concise, neutral summary (4-5 sentences) of the events that took place during the hour. This description should detail the observed movement patterns and sensor events without implying any evaluation.
    result: A field that is either "normal" or "anomalous" based strictly on the log data. Do not embed this judgment in the narrative.
    start_timestamp and end_timestamp: The exact timestamps from the logs that mark the beginning and end of the 6-hour interval.
    details: A list of the raw log entries that occurred during the interval. Each log entry should include the original details (e.g., {"timestamp": 1737900000, "room": "pillbox", "nodeId": 1, "onOff": true}) and be escaped appropriately for JSON formatting.

Important rules:

    Do not include any evaluative or judgmental language in the "situation_description." The narrative should allow a human or another LLM to independently assess whether the events are typical for an elderly person in the house.
    When describing movements and events, you should mention human readable times of these movements and events but ensure that times used are consistent with the provided start and end timestamps.
    When making a determination on whether this situation is normal or anomalous for an elderly person ensure that you consider the times of the day that would be normal for these types of movements and events for an elderly person.
    Ensure that any time mentioned in the situation description directly corresponds to the start or end timestamps (and any other provided timestamps), without introducing new or inconsistent values.
    Process and return only one 6-hour interval grouping per response.
    When determined whether this situation's result is normal or anomalous you should consider times of events such as very late night time and early morning hours being unusual for elderly people.

The output format must strictly adhere to the following JSON structure:
{
  "situations": [
    {
      "situation_description": "Neutral narrative describing the events that occurred.",
      "result": "normal | anomalous",
      "start_timestamp": "exact log entry timestamp marking the start",
      "end_timestamp": "exact log entry timestamp marking the end",
      "details": "list of log entries that occurred in the interval"
    }
  ]
}

"""

USER_PROMPT_PREFIX = """Analyze the following 6-hour block of log entries and create a neutral, human-readable scenario description of the events observed. 
Your description should summarize the recorded movement patterns in 4-5 clear sentences without including any evaluative language regarding whether the movement is 
typical or atypical. 
Here are the log entries:

"""

USER_PROMPT_SUFFIX = "\n\nStrictly respond in valid JSON format, and only JSON and include exactly 1 situation, no more."



In [67]:
# We are going to get the input data to inspect in 1 hour blocks

filename = "data/daily_routine_data.json"

INVESTIGATON_PERIOD_IN_HOURS = 6

def parse():
    """
    Parse the input file and group events by a 6-hour window.
    """
    if not os.path.exists(filename):
        return []

    with open(filename, 'r') as file:
        # Load and sort events by timestamp
        try:
            events = [json.loads(line) for line in file]
        except json.JSONDecodeError:
            print("error")
            return []

        events.sort(key=lambda x: datetime.fromtimestamp(x["timestamp"]))
    
        grouped = []
        window_start = None
        buffer = []
    
        # Group events in 6-hour windows
        for event in events:
            event_time = datetime.fromtimestamp(event["timestamp"])
            
            # Start or reset grouping window
            if window_start is None or event_time > window_start + timedelta(hours=INVESTIGATON_PERIOD_IN_HOURS):
                if buffer:
                    grouped.append({"details": buffer.copy()})
                buffer.clear()
                window_start = event_time
    
            buffer.append(event)
    
        # Append any remaining events in the buffer
        if buffer:
            grouped.append({"details": buffer})

        return grouped

    return []


In [68]:
loaded = parse()

# Lets take the latest hour
loaded = [loaded[2]] 
print('\n\n'.join([json.dumps(situation) for situation in loaded]))



{"details": [{"timestamp": 1737944400, "room": "kitchen", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}}, {"timestamp": 1737945600, "room": "livingroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737945660, "room": "livingroom", "nodeId": 1, "endpointId": 1, "attribute": {"OccupancySensing": {"Occupancy": 1}}}, {"timestamp": 1737949200, "room": "livingroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}}, {"timestamp": 1737952800, "room": "bedroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737952860, "room": "pillbox", "nodeId": 1, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737953400, "room": "bedroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}}]}


In [69]:
# The LLM appear to get confused with timestamped data so lets add a human readable time

# Iterate over each event in the details list
for event in loaded[0]["details"]:
    ts = event["timestamp"]
    # Convert timestamp to a datetime object in local timezone
    dt = datetime.fromtimestamp(ts, tz=TIMEZONE)
    # Format datetime to a human-readable string (e.g., "Sun Jan 26 2025 20:00:00")
    event["eventTime"] = dt.strftime("%a %b %d %Y %H:%M:%S")

# Print the updated JSON data structure
print(json.dumps(loaded[0]))

{"details": [{"timestamp": 1737944400, "room": "kitchen", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}, "eventTime": "Mon Jan 27 2025 02:20:00"}, {"timestamp": 1737945600, "room": "livingroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}, "eventTime": "Mon Jan 27 2025 02:40:00"}, {"timestamp": 1737945660, "room": "livingroom", "nodeId": 1, "endpointId": 1, "attribute": {"OccupancySensing": {"Occupancy": 1}}, "eventTime": "Mon Jan 27 2025 02:41:00"}, {"timestamp": 1737949200, "room": "livingroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}, "eventTime": "Mon Jan 27 2025 03:40:00"}, {"timestamp": 1737952800, "room": "bedroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}, "eventTime": "Mon Jan 27 2025 04:40:00"}, {"timestamp": 1737952860, "room": "pillbox", "nodeId": 1, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}, "eventTime": "Mon Jan 27 2025 04:41:00"}, {"timestamp": 17

In [70]:
def make_user_prompt(loaded) -> str:
    """
    Create a user prompt for OpenAI based on the scraped deals provided
    """
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([json.dumps(situation) for situation in loaded])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [71]:
make_user_prompt(loaded)

'Analyze the following 6-hour block of log entries and create a neutral, human-readable scenario description of the events observed. \nYour description should summarize the recorded movement patterns in 4-5 clear sentences without including any evaluative language regarding whether the movement is \ntypical or atypical. \nHere are the log entries:\n\n{"details": [{"timestamp": 1737944400, "room": "kitchen", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}, "eventTime": "Mon Jan 27 2025 02:20:00"}, {"timestamp": 1737945600, "room": "livingroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}, "eventTime": "Mon Jan 27 2025 02:40:00"}, {"timestamp": 1737945660, "room": "livingroom", "nodeId": 1, "endpointId": 1, "attribute": {"OccupancySensing": {"Occupancy": 1}}, "eventTime": "Mon Jan 27 2025 02:41:00"}, {"timestamp": 1737949200, "room": "livingroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}, "eventTime": "Mon Jan


## Structure Response using Pydantic Classes
We need this to tell how the LLM should respond with Structured Responses!!

In [72]:
from pydantic import BaseModel


class Situation(BaseModel):
    """
    A class to Represent a Situation with a summary description
    """
    situation_description: str
    result: str
    start_timestamp: int
    end_timestamp: int
    details: List[str]

class SituationSelection(BaseModel):
    """
    A class to Represent a list of Situations
    """
    situations: List[Situation]

class Investigation(BaseModel):
    """
    A class to represent a possible situation: a Situation where we estimate
    it could be anomalous
    """
    situation: Situation
    estimate: str


In [73]:
messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": make_user_prompt(loaded)}
    ]
print(messages)

[{'role': 'system', 'content': 'The timezone here is GMT. Your task is to analyze log entries from home sensors from a home occupied by an elderly person by grouping them into 6-hour intervals and converting each grouping into a clear, human-readable scenario. For each 6-hour interval, generate a neutral narrative that describes what occurred during that period. Important: The narrative (under "situation_description") must describe the events without any judgment about whether they are normal or anomalous. The determination of "normal" or "anomalous" should be provided solely in the separate "result" field when you are highly confident.\n\nFor each 6-hour grouping, ensure the output includes:\n\n    situation_description: A concise, neutral summary (4-5 sentences) of the events that took place during the hour. This description should detail the observed movement patterns and sensor events without implying any evaluation.\n    result: A field that is either "normal" or "anomalous" based

In [74]:
result = openai.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": make_user_prompt(loaded)}
    ],
    seed=42,
    response_format=SituationSelection
)

result = result.choices[0].message.parsed
result.situations = [situation for situation in result.situations if situation.result is not None]

In [75]:
print(result)

situations=[Situation(situation_description='During the early morning hours of January 27, 2025, there was activity recorded in various rooms. At 2:20 AM, the kitchen sensor indicated that an appliance was turned off. Shortly after, at 2:40 AM, a light was turned on in the living room, and occupancy was detected at 2:41 AM. The living room light was then turned off at 3:40 AM. Later, at 4:40 AM, a light in the bedroom was activated, followed by the pillbox sensor being activated at 4:41 AM, and this light was subsequently turned off at 4:50 AM.', result='anomalous', start_timestamp=1737944400, end_timestamp=1737953400, details=['{"timestamp": 1737944400, "room": "kitchen", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}, "eventTime": "Mon Jan 27 2025 02:20:00"}', '{"timestamp": 1737945600, "room": "livingroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}, "eventTime": "Mon Jan 27 2025 02:40:00"}', '{"timestamp": 1737945660, "room": "livi